In this notebook, I show how to create a star cluster with binary stars using AMUSE with MASC.
Below, I also show how this cluster can then be integrated using the Multiples module of AMUSE.
Please feel free to use this code if it's useful.

In [ ]:
# The following Python packages and their dependencies are required for this notebook
# You can find installation instructions for AMUSE at https://www.amusecode.org

# uncomment line below to install via pip - note that you must already have the dependencies!
#!pip install amuse-framework amuse-masc amuse-ph4 amuse-smalln amuse-kepler

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

from amuse.units import units, nbody_system, constants
from amuse.ext.masc.cluster import new_star_cluster

Setting up a star cluster, where the semi-major axis of the binaries is a constant.
It can also be an array (of same length as number_of_binaries).
More options are available (e.g. for mass distributions) and even more can/will be added in the future.

In [ ]:
np.random.seed(1701)
number_of_binaries = 100
number_of_systems = 5000
semi_major_axis = np.ones(number_of_binaries) * (10 | units.au)

single_stars, binary_stars, binary_systems = new_star_cluster(
    star_distribution='fractal',
    number_of_stars=number_of_systems,  # note that this really means 'number of systems'
    number_of_binaries=number_of_binaries,
    return_binaries=True,
    binary_semi_major_axis_distribution=semi_major_axis,
    eccentricity_distribution=0,
)

In [ ]:
print(
    f"Singles: {len(single_stars)}, "
    f"binary systems: {len(binary_systems)}, "
    f"total stars: {len(single_stars)+len(binary_stars)}"
)

In [ ]:
def plot_binary(b):
    l = (b[0].position - b[1].position).length()
    fig = plt.figure(figsize=(8, 8))
    ax_zoom = fig.add_subplot(111)
    ax_zoom.scatter(b.x.value_in(units.pc), b.y.value_in(units.pc), edgecolors=None, s=10)
    ax_zoom.set_xlim((b[0].x - l).value_in(units.pc), (b[0].x + l).value_in(units.pc))
    ax_zoom.set_ylim((b[0].y - l).value_in(units.pc), (b[0].y + l).value_in(units.pc))
    ax_zoom.quiver(
        b.x.value_in(units.pc), b.y.value_in(units.pc),
        b.vx.value_in(units.kms), b.vy.value_in(units.kms),
        scale=50
    )
    ax_zoom.set_aspect(1)
    return fig

def plot(s, b, width=5 | units.pc):
    length_unit = width.unit
    fig = plt.figure(figsize=(8, 16))
    ax = fig.add_subplot(121)
    ax.scatter(
        s.x.value_in(length_unit),
        s.y.value_in(length_unit),
        edgecolors=None, s=1,
    )
    ax.set_xlim(-width.number, width.number)
    ax.set_ylim(-width.number, width.number)
    ax.set_xlabel(f'x [{length_unit}]')
    ax.set_ylabel(f'y [{length_unit}]')
    ax.set_title('Fractal cluster')
    ax.set_aspect(1)
    ax_zoom = fig.add_subplot(122)
    
    n = s.nearest_neighbour()
    d = (n.position - s.position).lengths()

    zoom_in_mass = (b[0].mass + n[0].mass)
    zoom_in_position = (
        b[0].mass * b[0].position
        + n[0].mass * n[0].position
    ) / zoom_in_mass
    zoom_in_velocity = (
        b[0].mass * b[0].velocity
        + n[0].mass * n[0].velocity
    ) / zoom_in_mass
    system_projected_size = (
        (b[0].x - n[0].x)**2
        + (b[0].y - n[0].y)**2
    )**0.5

    ax_zoom.scatter(
        s.x.value_in(length_unit),
        s.y.value_in(length_unit),
        edgecolors=None, s=10,
        label='Center-of-mass',
    )
    ax_zoom.scatter(
        b.x.value_in(length_unit),
        b.y.value_in(length_unit),
        edgecolors=None, s=10,
        label='Component stars',
    )
    ax_zoom.set_xlim(
        (zoom_in_position.x - system_projected_size).value_in(length_unit),
        (zoom_in_position.x + system_projected_size).value_in(length_unit)
    )
    ax_zoom.set_ylim(
        (zoom_in_position.y - system_projected_size).value_in(length_unit),
        (zoom_in_position.y + system_projected_size).value_in(length_unit)
    )
    ax_zoom.quiver(
        zoom_in_position.x.value_in(length_unit),
        zoom_in_position.y.value_in(length_unit),
        (zoom_in_velocity.x).value_in(units.kms),
        (zoom_in_velocity.y).value_in(units.kms),
        scale=10,
    )
    ax_zoom.quiver(
        b.x.value_in(length_unit),
        b.y.value_in(length_unit),
        (b.vx - zoom_in_velocity.x).value_in(units.kms),
        (b.vy - zoom_in_velocity.y).value_in(units.kms),
        scale=10,
    )
    ax_zoom.set_aspect(1)
    ax_zoom.set_xlabel(f'x [{length_unit}]')
    ax_zoom.set_ylabel(f'y [{length_unit}]')
    ax_zoom.set_title('Zoom in on a binary pair')
    ax_zoom.legend()
    fig.tight_layout()
    return fig

In [ ]:
starplot = plot((binary_stars | single_stars), binary_stars)

In [ ]:
converter = nbody_system.nbody_to_si(
    single_stars.total_mass() + binary_stars.total_mass(),
    0.01 | units.Myr
)

In [ ]:
# We can simulate the cluster with the 'Ph4' code (Parallel Hermite 4th order),
# as long as the binaries aren't very tight.
# If they are, Ph4 will slow down to a crawl as the required timestep becomes too small.
# We then need to find another solution - like Multiples (below).
# The `number_of_workers` keyword indicates how many processes we will use for Ph4.
from amuse.community.ph4 import Ph4
grav = Ph4(converter, number_of_workers=6)
s_in_grav = grav.particles.add_particles(single_stars)
b_in_grav = grav.particles.add_particles(binary_stars)

time_start = time.time() | units.s
grav.evolve_model(0.001 | units.Myr)
time_end = time.time() | units.s
print(f"This took {time_end-time_start}")

# A channel is a way to copy updated properties from an in-code particle set back to our model.

channel = b_in_grav.new_channel_to(binary_stars)
channel.copy()
channel = s_in_grav.new_channel_to(single_stars)
channel.copy()
grav.stop()

In [ ]:
# Note how the binary progressed in its orbit while the cluster as a whole is basically unchanged
# - this illustrates how much the timescales differ.
starplot = plot(binary_stars | single_stars, binary_stars)

In [ ]:
# Now we simulate the system further using Multiples.
# This module uses Ph4 for the integration of the systems, 
# but integrates the binaries using a Kepler integrator
# and interactions between stars and binary systems using a small-N integrator
# As a result, it can speed things up by a large factor when dealing with tight binaries.
# When this is not the case however, it can be much slower!
from amuse.community.ph4 import Ph4
from amuse.community.smalln import Smalln
from amuse.community.kepler import Kepler
import amuse.couple.multiples as multiples


# Multiples requires some setting up - you can use this class for that
class GravityWithMultiples:
    def __init__(
        self, stars,
        gravity=Ph4, smalln=Smalln, kepler=Kepler,
        converter=None, time_step=0.01 | units.Myr,
        number_of_workers=1,
    ):
        self.smalln_code = smalln
        if converter is None:
            self.converter = nbody_system.nbody_to_si(stars.total_mass(), time_step)
        else:
            self.converter = converter
        self.model_time = 0 | units.Myr

        self.smalln = None
        self.gravity = gravity(convert_nbody=self.converter, number_of_workers=number_of_workers)
        self.gravity.parameters.epsilon_squared = (self.converter.to_si(0.0 | nbody_system.length))**2
        self.gravity.particles.add_particles(stars)

        stopping_condition = self.gravity.stopping_conditions.collision_detection
        stopping_condition.enable()

        self.init_smalln()
        self.kepler = kepler(unit_converter=self.converter)
        self.kepler.initialize_code()
        self.multiples_code = multiples.Multiples(
            self.gravity, self.new_smalln, self.kepler, constants.G
        )
        self.multiples_code.neighbor_perturbation_limit = 0.05
        self.multiples_code.global_debug = 1

        #  global_debug = 0: no output from multiples
        #                 1: minimal output
        #                 2: debugging output
        #                 3: even more output

        print('Settings for Multiples:')
        print(f'multiples_code.neighbor_veto = {self.multiples_code.neighbor_veto}')
        print(f'multiples_code.neighbor_perturbation_limit = {self.multiples_code.neighbor_perturbation_limit}')
        print(f'multiples_code.retain_binary_apocenter = {self.multiples_code.retain_binary_apocenter}')
        print(f'multiples_code.wide_perturbation_limit = {self.multiples_code.wide_perturbation_limit}')

        self.energy_0 = self.print_diagnostics(self.multiples_code)

    @property
    def stars(self):
        return self.multiples_code.stars

    @property
    def particles(self):
        return self.multiples_code.particles

    def init_smalln(self):
        self.smalln = self.smalln_code(convert_nbody=self.converter)

    def new_smalln(self):
        self.smalln.reset()
        return self.smalln

    def stop_smalln(self):
        self.smalln.stop()

    def print_diagnostics(self, grav, energy_0=None):
        # Simple diagnostics.
        energy_kinetic = grav.kinetic_energy
        energy_potential = grav.potential_energy
        (
            self.number_of_multiples,
            self.number_of_binaries,
            self.energy_in_multiples,
        ) = grav.get_total_multiple_energy()
        energy = energy_kinetic + energy_potential + self.energy_in_multiples
        print(f'Time = {grav.get_time().in_(units.Myr)}')
        print(f'    top-level kinetic energy = {energy_kinetic}')
        print(f'    top-level potential energy = {energy_potential}')
        print(f'    total top-level energy = {energy_kinetic + energy_potential}')
        print(f'    {self.number_of_multiples} multiples, total energy = {self.energy_in_multiples}')
        print(f'    uncorrected total energy ={energy}')

        energy_tidal = (
            grav.multiples_external_tidal_correction
            + grav.multiples_internal_tidal_correction
        )  # tidal error
        energy_error = grav.multiples_integration_energy_error  # integration error

        energy -= energy_tidal + energy_error
        print(f'    corrected total energy = {energy}')

        if energy_0 is not None: print('    relative energy error=', (energy-energy_0)/energy_0)

        return energy

    def evolve_model(self, t_end):
        self.multiples_code.evolve_model(t_end)
        self.print_diagnostics(self.multiples_code, self.energy_0)
        self.model_time = self.multiples_code.model_time

    def stop(self):
        self.gravity.stop()
        self.kepler.stop()
        self.stop_smalln()

In [ ]:
# Now we actually run with Multiples
grav = GravityWithMultiples(
    binary_stars | single_stars,
    gravity=Ph4, smalln=Smalln, kepler=Kepler,
    converter=converter,
    number_of_workers=6,
)
time_start = time.time() | units.s
grav.evolve_model(0.01 | units.Myr)
time_end = time.time() | units.s
print(f"This took {time_end-time_start}.")
channel = grav.particles.new_channel_to(binary_stars)
channel.copy()
channel = grav.particles.new_channel_to(single_stars)
channel.copy()
grav.stop()

In [ ]:
# Let's plot the system and the binary again
starplot = plot(binary_stars | single_stars, binary_stars)

Below are some potentially useful methods/functions, without much explanation - mostly here to remember that they exist. Some are very slow so execute with caution!

In [ ]:
# the 'get_binaries' method will search for binaries the hard way - use
# with caution and probably never use it on a large particle set!
# it will only return binaries of a specified (or default) 'hardness'

stars_in_ph4 = grav.particles
q = stars_in_ph4.get_binaries()  # control via 'hardness' keyword
print(len(q))

In [ ]:
# use this function to extract the orbital parameters from a binary.
# the module has other useful functions for dealing with binaries.
from amuse.ext.orbital_elements import get_orbital_elements_from_binary

mass1, mass2, a, e, c_o_ta, c_o_inc, c_o_lotan, c_o_aop = get_orbital_elements_from_binary(
    binary_stars[0] + binary_stars[number_of_binaries]
)
print(mass1, mass2, a.in_(units.au), e)